In [ ]:
import os
import openai
import numpy as np
import time
import traceback
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [ ]:
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_base = os.environ.get("OPENAI_API_BASE")

In [ ]:
chat_completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Please tell me a shot story (3 paragraphs) about a speaking strawberry, a very shy giant and a tiny wizard with a giant hat."}],
    stream=False,
    temperature=0.7,
)

In [ ]:
if isinstance(chat_completion, dict):
    # not stream
    print(chat_completion.choices[0].message.content)
else:
    # stream
    for token in chat_completion:
        content = token["choices"][0]["delta"].get("content")
        if content != None:
            print(content, end="", flush=True)

In [ ]:
print(chat_completion["usage"])

In [ ]:
pricing_table = {
    "gpt-3.5": {"input": 0.001 / 1000, "output": 0.002 / 1000},
    "gpt-4": {"input": 0.03 / 1000, "output": 0.06 / 1000},
    "gpt-4-0613": {"input": 0.03 / 1000, "output": 0.06 / 1000},
    "gpt-4-1106-preview": {"input": 0.01 / 1000, "output": 0.03 / 1000},
}
models = [
    "gpt-3.5-turbo",
    "gpt-3.5-turbo-0613",
    "gpt-3.5-turbo-1106",
    "gpt-3.5-turbo-16k",
    "gpt-3.5-turbo-16k-0613",
    "gpt-4",
    "gpt-4-0613",
    "gpt-4-1106-preview"
]

In [ ]:
def prompt_api(model, prompt):
    print(f"Using model {model}")
    for i in range(10):
        try:
            chat_completion = openai.ChatCompletion.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            stream=False,
            # max_tokens=100,
            )
            if isinstance(chat_completion, dict):
                # not stream
                print(chat_completion.choices[0].message.content)
            else:
                # stream
                for token in chat_completion:
                    content = token["choices"][0]["delta"].get("content")
                    if content != None:
                        print(content, end="", flush=True)
            n_input_tokens = int(chat_completion["usage"]["prompt_tokens"])
            n_output_tokens = int(chat_completion["usage"]["completion_tokens"])
            if "gpt-3.5" in model:
                model_name = "gpt-3.5"
            else:
                model_name = model
            input_price = pricing_table[model_name]["input"]
            output_price = pricing_table[model_name]["output"]
            print(f"Total number of tokens: {chat_completion['usage']['total_tokens']}")
            print(f"price: {np.round(n_input_tokens * input_price + n_output_tokens * output_price, 6)}$")
            break
        except Exception as exc:
            print(traceback.format_exc())
            print(exc)
            if i != 9:
                print(f"Retrying... (i = {i})")
            time.sleep(3)

In [ ]:
prompt = "Can you quickly summarize the 3 most important news of the day?"
for model in models:
    prompt_api(model, prompt)
    print("")
    print("")
    print("")

In [ ]:
# prompt_api("gpt-4-1106-preview", "Can you repeat to me the last thing i asked you?")
prompt_api("gpt-4-1106-preview", "Can you repeat to me the last thing i asked you, as well as the thing i asked you efore that?")